In [742]:
import pandas as pd
import time


class Timer:
    def __init__(self, name=None):
        self.name = name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time_ns() / 1000
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        end_time = time.time_ns() / 1000  # Record end time
        elapsed = end_time - self.start_time
        if self.name:
            print(f"[{self.name}] Elapsed time: {elapsed:.2f} milliseconds")
        else:
            print(f"Elapsed time: {elapsed:.2f} milliseconds")

### 1. Lecture CSV et renommage

In [743]:
# https://catalog.data.gov/dataset/meteorite-landings
df_meteorites = pd.read_csv("data/nasa_meteorites.csv")

df_meteorites.columns

Index(['name', 'id', 'nametype', 'recclass', 'mass (g)', 'fall', 'year',
       'reclat', 'reclong', 'country_id', 'GeoLocation'],
      dtype='object')

In [744]:
# On a les long / lat déjà, on supprime la colonne GeoLocation
df_meteorites = df_meteorites.drop("GeoLocation", axis=1)

In [745]:
# On renomme en snake_case les colonnes qui sont mal nommées
df_meteorites = df_meteorites.rename(columns={
    "nametype": "name_type",
    "recclass": "rec_class",
    "mass (g)": "mass_g",
    "reclat": "latitude",
    "reclong": "longitude",
})

### 2. Validation des données (qualité)

In [746]:
# On valide que les latitude / longitude ne sont pas nulles
df_meteorites[(df_meteorites.latitude.isna()) | df_meteorites.longitude.isna()]

,name,id,name_type,rec_class,mass_g,fall,year,latitude,longitude,country_id
147,Bulls Run,5163,Valid,Iron?,2250.000,Fell,1964.0,NaN,NaN,NaN
208,Clohars,5383,Valid,L4,48.600,Fell,1822.0,NaN,NaN,NaN
409,Jalanash,12068,Valid,Ureilite,700.000,Fell,1990.0,NaN,NaN,NaN
414,Jemlapur,12079,Valid,L6,450.000,Fell,1901.0,NaN,NaN,NaN
520,Cumulus Hills 04075,32531,Valid,Pallasite,9.600,Found,2003.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
44000,Yamato 981086,37708,Valid,H4,5.227,Found,1998.0,NaN,NaN,NaN
44001,Yamato 981090,37712,Valid,H4,8.682,Found,1998.0,NaN,NaN,NaN
45589,Yamato 984028,40648,Valid,Martian (shergottite),12.342,Found,1998.0,NaN,NaN,NaN
45660,Yambo no. 2,30346,Valid,L3,3.200,Found,1975.0,NaN,NaN,NaN


In [747]:
# On vérifie s'il y a des duplicats dans les noms de météorites
df_meteorites[df_meteorites.name.duplicated()]

,name,id,name_type,rec_class,mass_g,fall,year,latitude,longitude,country_id


In [748]:
# On explore les différentes valeurs de la colonne name_type
df_meteorites.name_type.value_counts()

name_type
Valid     45641
Relict       75
Name: count, dtype: int64

### 3. La performance et les index

In [749]:
# Regardons ça prend combien de millisecondes pour aller chercher une ligne précise du df
with Timer():
    df_meteorites_455 = df_meteorites[df_meteorites["id"] == 455]

Elapsed time: 753.00 milliseconds


In [750]:
df_meteorites.set_index("id").sort_index()

with Timer():
    df_meteorites_455_02 = df_meteorites[df_meteorites["id"] == 455]

Elapsed time: 275.00 milliseconds


### 4. JOIN avec les pays

In [751]:
# On lit un fichier JSON (fichier hiérarchique)
df_countries = pd.read_json("data/countries.json")
df_countries.rename(columns={"name": "country_name", "id": "_country_id"}, inplace=True)

df_countries.dtypes

_country_id      int64
country_name    object
dtype: object

In [752]:
# On veut join aux pays, mais on ne veut pas perdre de lignes de météorites si le pays n'existe pas (LEFT JOIN)
df_meteorites = df_meteorites.merge(df_countries, how="left", left_on="country_id", right_on="_country_id")

### 5. Corrigeons les données

In [753]:
df_meteorites

,name,id,name_type,rec_class,mass_g,fall,year,latitude,longitude,country_id,_country_id,country_name
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,38.0,38.0,Germany
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,31.0,31.0,Denmark
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,20.0,20.0,Canada
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,67.0,67.0,Mexico
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,4.0,4.0,Argentina
...,...,...,...,...,...,...,...,...,...,...,...,...
45711,Zillah 002,31356,Valid,Eucrite,172.0,Found,1990.0,29.03700,17.01850,61.0,61.0,Libya
45712,Zinder,30409,Valid,"Pallasite, ungrouped",46.0,Found,1999.0,13.78333,8.96667,75.0,75.0,Niger
45713,Zlin,30410,Valid,H4,3.3,Found,1939.0,49.25000,17.66667,29.0,29.0,Czech Republic
45714,Zubkovsky,31357,Valid,L6,2167.0,Found,2003.0,49.78917,41.50460,90.0,90.0,Russia



- Lire fichier CSV
- Renommage (renommer en snake_case) DONE
- Index et performance
- Valider la qualité de données (checker si ya des long/lat nulles, checker enum) DONE
- JOIN
- Colonnes dérivées (calculer kg, calculer continent NorthAmerica boolean, calculer old, really old)
- Filtres
- Agrégations





### 6. Dériver des colonnes

In [754]:
# Dérivons le poids du météorite en KGs
df_meteorites["mass_kg"] = df_meteorites["mass_g"] / 1000

In [755]:
# Dérivons le booléen is_north_american
df_meteorites["is_in_north_america"] = df_meteorites.apply(lambda row: row["country_name"] in ["United States of America", "Mexico", "Canada"], axis=1)

In [756]:
# Dérivons le critère d'ancienneté de la découverte du météorite en 3 catégories: "recent", "old", "very old"
df_meteorites['era_group'] = pd.cut(df_meteorites['year'], bins=3, labels=['Ancient', 'Old', 'Recent'])

### 7. Corriger / Filtrer les données

In [757]:
# Filtrer les lignes qui ont une valeur nulle dans longitude ou latitude ou une valeur nulle dans country_name
df_meteorites = df_meteorites.dropna(subset=['latitude', 'longitude', 'country_name'])

In [758]:
# Filtrer les lignes qui ont une valeur 0 dans longitude ou latitude
df_meteorites = df_meteorites[(df_meteorites['latitude'] != 0) & (df_meteorites['longitude'] != 0)]

### 8. Agrégations

In [759]:
# Grouper les lignes par pays, et calculer:
# - l'année la plus ancienne (oldest_year)
# - le nombre de météorites (meteorite_count)
# - le poid moyen des météorites en grammes (mean_weight)
df_grouped_by_country = df_meteorites.groupby("country_name").agg(
    oldest_year=("year", "min"),
    meteorite_count=("name", "count"),
    mean_weight=("mass_g", "mean"),
)

### 9. Écrire le dataframe agrégé (grouped) dans un fichier Excel


In [760]:
df_grouped_by_country.to_excel("data/meteorites_grouped_by_country.xlsx", sheet_name="agregated")